## Deliverable 2. Create a Customer Travel Destinations Map.

In [16]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [17]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Yerbogachen,RU,61.2767,108.0108,-44.25,78,47,5.50,scattered clouds
1,1,Bronnoysund,NO,65.4625,12.1997,30.20,69,75,12.66,light snow
2,2,Rikitea,PF,-23.1203,-134.9692,80.11,67,88,7.54,overcast clouds
3,3,New Norfolk,AU,-42.7826,147.0587,66.99,87,54,1.01,broken clouds
4,4,Kudahuvadhoo,MV,2.6708,72.8944,81.07,78,67,15.57,broken clouds


In [18]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 90


In [19]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df['Max Temp'] >= min_temp) & (city_data_df['Max Temp'] <= max_temp)]
preferred_cities_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
2,2,Rikitea,PF,-23.1203,-134.9692,80.11,67,88,7.54,overcast clouds
4,4,Kudahuvadhoo,MV,2.6708,72.8944,81.07,78,67,15.57,broken clouds
8,8,Quelimane,MZ,-17.8786,36.8883,82.40,89,75,5.75,broken clouds
15,15,Villa Carlos Paz,AR,-31.4241,-64.4978,84.00,75,28,10.00,scattered clouds
16,16,Puerto Ayora,EC,-0.7393,-90.3518,84.20,70,40,14.97,light rain


In [20]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()

City_ID                197
City                   197
Country                197
Lat                    197
Lng                    197
Max Temp               197
Humidity               197
Cloudiness             197
Wind Speed             197
Current Description    197
dtype: int64

In [21]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
# No empty rows found

In [22]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = preferred_cities_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
2,Rikitea,PF,80.11,overcast clouds,-23.1203,-134.9692,
4,Kudahuvadhoo,MV,81.07,broken clouds,2.6708,72.8944,
8,Quelimane,MZ,82.40,broken clouds,-17.8786,36.8883,
15,Villa Carlos Paz,AR,84.00,scattered clouds,-31.4241,-64.4978,
16,Puerto Ayora,EC,84.20,light rain,-0.7393,-90.3518,
17,Arraial Do Cabo,BR,82.40,clear sky,-22.9661,-42.0278,
18,Kolondieba,ML,76.06,broken clouds,11.0882,-6.8926,
20,Maningrida,AU,78.35,overcast clouds,-12.0667,134.2667,
25,Ahuimanu,US,78.80,broken clouds,21.4447,-157.8378,
26,Kapaa,US,75.20,scattered clouds,22.0752,-159.3190,


In [34]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
# 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"] = f"{lat},{lng}"
# 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

# 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()

# 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print('Hotel not found... skipping.')

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [41]:
# 7. Drop the rows where there is no Hotel Name.
hotel_df = hotel_df[(hotel_df['Hotel Name'] != "")]

In [42]:
# 8a. Create the output File (CSV)
output_data_file = 'WeatherPy_vacation.csv'

# 8b. Export the City_Data into a csv
hotel_df.to_csv(output_data_file, index_label="City_ID")

In [48]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Curent Weather</dt><dd>{Current Description} and {Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = hotel_df[["Lat", "Lng"]]

In [50]:
# 11a. Add a marker layer for each city to the map. 
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,
             dissipating=False, max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)
# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))